<a href="https://colab.research.google.com/github/anhnguyenvv/NLP-A/blob/main/_Lab_1_Train_1_LLM_ipynb_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1: Working with Text



> Họ tên: Nguyễn Thị Lan Anh </br>
> MSSV: 21120198

# Bài tập 1:

Tiến hành train một LLM dựa trên GPT2 theo hướng dẫn dưới đây


In [ ]:
!pip install tiktoken
!pip install torch

Packages that are being used in this notebook:

In [ ]:
from importlib.metadata import version

import tiktoken
import torch

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.2.1+cu121
tiktoken version: 0.6.0


In [ ]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={'<|endoftext|>'})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

    return dataloader

- Load raw text we want to work with

In [ ]:

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [ ]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [ ]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
block_size = max_length
pos_embedding_layer = torch.nn.Embedding(block_size, output_dim)

In [ ]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


# Bài tập 2:

Thực hiện và ghi kết quả chi tiết các bước encode và decode chuỗi sau: "Akwirw ier"



In [ ]:
# Bài tập 2:
tokenizer = tiktoken.get_encoding("gpt2")
text = "Akwirw ier"

> Encode


In [ ]:
nums = tokenizer.encode(text)
print(nums)

[33901, 86, 343, 86, 220, 959]


> Decode

In [ ]:
tokenizer.decode(nums)

'Akwirw ier'

> Các bước decode

- Lần lượt decode từng token ID

In [ ]:
tokens = []
for num in nums:
   tokens.append(tokenizer.decode([num]))
   print(f"{num} -> '{tokens[-1]}'")

33901 -> 'Ak'
86 -> 'w'
343 -> 'ir'
86 -> 'w'
220 -> ' '
959 -> 'ier'


- Ghép các tokens lại thành text

In [ ]:
text = "".join(t for t in tokens)
print(text)

Akwirw ier


> Các bước encode

- Ta có hàm encode tách text thành các tokens là các từ phụ hoặc ký tự riêng lẻ:

In [ ]:
print(f"'{text}' ->", end = " ")
print(tokens)

'Akwirw ier' -> ['Ak', 'w', 'ir', 'w', ' ', 'ier']


- chuyển các tokens thành ID

In [ ]:
IDs = []
for token in tokens:
   IDs+=(tokenizer.encode(token))
   print(f"'{token}' -> {IDs[-1]}")


'Ak' -> 33901
'w' -> 86
'ir' -> 343
'w' -> 86
' ' -> 220
'ier' -> 959


- thu được

In [ ]:
print(IDs)

[33901, 86, 343, 86, 220, 959]
